In [ ]:
import fasttext
import numpy as np
import json
import random
import datetime
import os

Via https://fasttext.cc

```
wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.it.zip
```

In [ ]:
wikimodel = fasttext.FastText.load_model("data/wiki.it/wiki.it.bin")

ccmodel = fasttext.FastText.load_model("data/cc.it.300.bin")

In [ ]:
def cosine_distance(first, second, model, decimals=2):
    exact_distance = (np.dot(model[first], model[second])
            /(np.linalg.norm(model[first])*np.linalg.norm(model[second]))
           )
    truncated_distance = np.round(float(exact_distance*100),decimals)
    return truncated_distance

## Extract Apple dictionary

https://gist.github.com/josephg/5e134adf70760ee7e49d

In [ ]:
# Thanks to commenters for providing the base of this much nicer implementation!
# Save and run with $ python 0dedict.py
# You may need to hunt down the dictionary files yourself and change the awful path string below.
# This works for me on MacOS 10.14 Mohave

from struct import unpack
from zlib import decompress
import re
# filename = '/System/Library/Assets/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/9f5862030e8f00af171924ebbc23ebfd6e91af78.asset/AssetData/Oxford Dictionary of English.dictionary/Contents/Resources/Body.data'
# filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/bad4a5c5bf10a2dd135a07d7ad7b28e23d0aa7dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"
filename = "/System/Library/AssetsV2/com_apple_MobileAsset_DictionaryServices_dictionaryOSX/a1fa9ccfe47b097a31a6b2ea94641e6e511855dd.asset/AssetData/Italian.dictionary/Contents/Resources/Body.data"

f = open(filename, 'rb')

def gen_entry():
    f.seek(0x40)
    limit = 0x40 + unpack('i', f.read(4))[0]
    f.seek(0x60)
    while f.tell()<limit:
        sz, = unpack('i', f.read(4))
        buf = decompress(f.read(sz)[8:])

        pos = 0
        while pos < len(buf):
            chunksize, = unpack('i', buf[pos:pos+4])
            pos += 4

            entry = buf[pos:pos+chunksize]
            title = re.search('d:title="(.*?)"', entry.decode('utf-8')).group(1)
            yield title, entry

            pos += chunksize

wordlist = []
for word, definition in gen_entry():
    wordlist.append(word)

In [ ]:
print(len(wordlist))

# Parole italiane

```
git clone https://github.com/napolux/paroleitaliane.git

sudo apt-get install witalian

/usr/share/dict/italian
```

In [ ]:
parole60  = "data/paroleitaliane/paroleitaliane/60000_parole_italiane.txt"
parole95  = "data/paroleitaliane/paroleitaliane/95000_parole_italiane_con_nomi_propri.txt"

parole280 = "data/paroleitaliane/paroleitaliane/280000_parole_italiane.txt"

paroleDebian = "data/italian"

# with open(paroleDebian, "r") as file:
#     words = [w.strip() for w in file.readlines() if "'" not in w]
    


In [ ]:
words = [w for w in wordlist if len(w)>1 and '-' not in w and '(' not in w and ')' not in w and "'" not in w]

In [ ]:
words

In [ ]:
len(words)

In [ ]:
with open("data/paroleitaliane/paroleitaliane/1000_parole_italiane_comuni.txt", "r") as file:
    common_words = [w.strip() for w in file.readlines()]
    
common_words = [w for w in common_words if len(w)>2]
len(common_words)

In [ ]:
def create_semantle_data(all_words, solution_words, model, date, puzzle_number, verbose=False):
    # Day to use for the solution
    isoday = date.isoformat()
    
    # Solution
    # Seed the random number generator
    random.seed(a=isoday, version=2)
    # Pick the word
    query = random.choice(solution_words)
    if verbose:
        print("La soluzione per il {} è \"{}\"".format(isoday, query))
        
    # Distanze
    distances_list = [(word, cosine_distance(query, word, model)) for word in set(all_words+solution_words) if len(word)>1]
    
    # Distanze nell'ordine giusto
    distances_list_sorted = sorted(distances_list, key=lambda x: x[1],reverse=True)
    
    # Database per Semantle
    
    word_database = {
        _tuple[0]:
        {
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted)
    }
    
    # Le parole più vicine
    closest_words_list = [
        {
            "w": _tuple[0],
            "s": _tuple[1],
            "r": _ix,
        } for _ix, _tuple in enumerate(distances_list_sorted[:1000])
    ]
    
    solution_word = query
    
    # Statistiche
    
    nearest_word_similarity = closest_words_list[1]["s"]
    tenth_nearest_word_similarity = closest_words_list[9]["s"]
    thousandth_nearest_word_similarity = closest_words_list[999]["s"]

    day_stats = {
        "puzzle_number": puzzle_number,
        "nearest_word_similarity": nearest_word_similarity,
        "tenth_nearest_word_similarity": tenth_nearest_word_similarity,
        "thousandth_nearest_word_similarity": thousandth_nearest_word_similarity,
    }

    semantle_data = {
        "word_database": word_database,
        "closest_words_list": closest_words_list,
        "solution_word": solution_word,
        "day_stats": day_stats,
    }
    return semantle_data

In [ ]:
wikidata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

ccdata = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date.today()-datetime.timedelta(days=2),
    puzzle_number=-1,
    verbose=True
)

In [ ]:
print([w['w'] for w in wikidata['closest_words_list'][:50]])
# print(wikidata['word_database']['re'])

In [ ]:
print([w['w'] for w in ccdata['closest_words_list'][:50]])
# print(ccdata['word_database']['re'])

In [ ]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=wikimodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)

In [ ]:
data = create_semantle_data(
    all_words=words,
    solution_words=common_words,
    model=ccmodel,
    date=datetime.date(2022,9,11),
    puzzle_number=1,
    verbose=True
)
print(data["word_database"]["conte"])
print([n["w"] for n in data["closest_words_list"]])


In [ ]:
for days_ahead in range(365):
    _date = datetime.date(2022,9,9)+datetime.timedelta(days=days_ahead)
    data = create_semantle_data(
        all_words=words,
        solution_words=common_words,
        model=ccmodel,
        date=_date,
        puzzle_number=days_ahead,
        verbose=True
    )
    isopath = _date.isoformat()
    os.makedirs("AppleCC/"+isopath, exist_ok = True)
    with open("AppleCC/"+isopath+"/semantle.json", "w") as file:
        json.dump(data,file)
    with open("AppleCC/"+isopath+"/closest.json", "w") as file:
        json.dump(data['closest_words_list'],file)
    with open("AppleCC/"+isopath+"/stats.json", "w") as file:
        json.dump(data['day_stats'],file)
    with open("AppleCC/"+isopath+"/database.json", "w") as file:
        json.dump(data['word_database'],file)

In [ ]:
wikidata.keys()